# Multi-View-Majority-Vote-Learning-Algorithms-Direct-Minimization-of-PAC-Bayesian-Bounds

This Notebook contains everything necessary to reproduce the experiments in our paper:  

*Multi-View Majority Vote Learning Algorithms: Direct Minimization of PAC-Bayesian Bounds*

## Imports

In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from collections import OrderedDict

# Scikit-learn
from sklearn import preprocessing
from sklearn.utils import check_random_state
RAND = check_random_state(42)

# torch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
from torch.autograd import Variable
from torch.utils.data import DataLoader,Dataset
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


from mvpb.dNDF import MultiViewBoundsDeepNeuralDecisionForests


# Import data
from mvlearn.datasets import load_UCImultifeature
from data.datasets import (SampleData,
                           Nutrimouse,
                           train_test_split,
                           train_test_merge,
                           s1_s2_split)
from mvpb.util import uniform_distribution

In [12]:
v=6
m=3

In [13]:
emp_risks_views  = [[0.43455497, 0.50580645, 0.60298103],
                                    [0.43682796, 0.6295302,  0.54495159],
                                    [0.51780822, 0.44562334, 0.56420765],
                                    [0.33877551, 0.23584906, 0.42354533],
                                    [0.44179523, 0.44889503, 0.45938748],
                                    [0.44385027, 0.53288591, 0.4295393 ]]


## Load and prepare the multiview datasets

In [3]:
mv_data, labels = load_UCImultifeature()
np.unique(labels)

array([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.])

In [4]:
scaler = preprocessing.MinMaxScaler()
for i in range(len(mv_data)):
    mv_data[i] = scaler.fit_transform(mv_data[i])

In [5]:
mvb = MultiViewBoundsDeepNeuralDecisionForests(nb_estimators=3, nb_views=len(mv_data), depth =3,used_feature_rate=0.8)

In [6]:
mvb.fit(mv_data,labels)

MultiViewBoundsDeepNeuralDecisionForests(depth=3, nb_estimators=3, nb_views=6,
                                         posterior=array([0.33333333, 0.33333333, 0.33333333]),
                                         used_feature_rate=0.8)

In [7]:
posterior_Qv , posterior_rho = mvb.optimize_rho('Lambda',delta=0.05, eps=10**-9)

[[0.43455497 0.50580645 0.60298103]
 [0.43682796 0.6295302  0.54495159]
 [0.51780822 0.44562334 0.56420765]
 [0.33877551 0.23584906 0.42354533]
 [0.44179523 0.44889503 0.45938748]
 [0.44385027 0.53288591 0.4295393 ]]
Bound: tensor(0.9570, dtype=torch.float64, grad_fn=<AddBackward0>)
Upd: tensor(0.9570, dtype=torch.float64, grad_fn=<AddBackward0>)
Différence: tensor(2.0000e-09, dtype=torch.float64, grad_fn=<SubBackward0>)
Iteration 0, Loss: 0.5747438456170645
Iteration 1, Loss: 0.5747322296035198
Iteration 2, Loss: 0.5747102462809931
Iteration 3, Loss: 0.574678791371241
Iteration 4, Loss: 0.5746388876374036
Iteration 5, Loss: 0.5745913540608433
Iteration 6, Loss: 0.5745369077830766
Iteration 7, Loss: 0.5744762641871223
Iteration 8, Loss: 0.5744100322808807
Iteration 9, Loss: 0.5743387077145884
Iteration 10, Loss: 0.5742628021420889
Iteration 11, Loss: 0.5741827709945986
Iteration 12, Loss: 0.574098983688286
Iteration 13, Loss: 0.574011809863725
Iteration 14, Loss: 0.5739215077232059
Ite

In [8]:
print(posterior_Qv[0])
print(posterior_Qv[1])
print(posterior_Qv[2])
print(posterior_Qv[3])
print(posterior_Qv[4])
print(posterior_Qv[5])

Parameter containing:
tensor([0.3761, 0.3374, 0.2865], requires_grad=True)
Parameter containing:
tensor([0.3865, 0.2850, 0.3285], requires_grad=True)
Parameter containing:
tensor([0.3285, 0.3675, 0.3040], requires_grad=True)
Parameter containing:
tensor([0.3288, 0.3928, 0.2785], requires_grad=True)
Parameter containing:
tensor([0.3379, 0.3340, 0.3281], requires_grad=True)
Parameter containing:
tensor([0.3468, 0.2983, 0.3548], requires_grad=True)


In [9]:
posterior_rho

Parameter containing:
tensor([0.0882, 0.0540, 0.0956, 0.4107, 0.1910, 0.1605], requires_grad=True)

In [11]:
vecteur = torch.tensor([-0.0944,  0.3723,  0.0580,  0.2210,  0.2957,  0.1475])

# Calcul du softmax
softmax_vecteur = torch.softmax(vecteur, dim=0)

print(softmax_vecteur)

tensor([0.1269, 0.2024, 0.1478, 0.1739, 0.1874, 0.1616])
